#### 1. 라이브러리 호출

In [14]:
import sys
sys.path.append('../')

In [15]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import model.metric as module_metric

from data_loader.dataloader import get_dataloader
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from model.loss import *
from train import *
from pathlib import Path


#### 2. 시드고정

In [16]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

#### 3. 하이퍼 파라미터 설정

In [17]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 16
num_epoch = 200
damage = 'dent'

train_dir = f'./dataset/{damage}/train/'
val_dir = f'./dataset/{damage}/valid/'
test_dir = f'./dataset/{damage}/test/'

mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)
image_size = 512

In [18]:
transform_train = A.Compose([
    A.Resize(image_size, image_size),
    A.HorizontalFlip(),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.Rotate((-10, 10), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_test = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(mean, std),
    transforms.ToTensorV2(transpose_mask=True)
])

In [19]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size, True)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size, False)
test_dataloader = get_dataloader(test_dir, transform_test, batch_size, False)

In [20]:
model = smp.Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)
model = model.to(device)

In [21]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=lr)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=20, T_mult=2, eta_min=1e-5)
metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

#### 4. wandb config 생성

In [22]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch
train_config['Image size'] = image_size

train_config['Loss fn'] = criterion.__class__.__name__
train_config['Optimizer'] = optimizer.__class__.__name__
train_config['LR Scheduler'] = scheduler.__class__.__name__
train_config['Metric'] = {str(idx+1) : metric for idx, metric in enumerate([metrics[i].__name__ for i in range(len(metrics))])}

In [23]:
save_dir = f"./saved/U-Net/{'_'.join([model.__dict__['name'].split('-')[1:][0].capitalize(), model.__dict__['name'].split('-')[-1].capitalize()])}_{damage}/"

trainer = Trainer(model, criterion, metrics, optimizer, device, num_epoch, save_dir, mean, std,
                  data_loader=train_dataloader, valid_data_loader=val_dataloader, test_data_loader=test_dataloader,
                  lr_scheduler=scheduler)

In [24]:
trainer.early_stop = 30
train_config['Early stop'] = trainer.early_stop

In [25]:
wandb.init(project=trainer.dir.split('/')[1], name=f"{trainer.dir.split('/')[2]}_train", config=train_config)

In [26]:
trainer.train()


Epoch : 0 | Train Loss : 0.21422 | Train P.A : 93.91% | Train IOU : 0.00000 | Val Loss : 0.07484 | Val P.A : 99.07% | Val IOU : 0.00000 | Training Time : 122.82sec


Train IOU,▁
Train Loss,▁
Train P.A,▁
Val IOU,▁
Val Loss,▁
Val P.A,▁
Train IOU,0.0
Train Loss,0.21422
Train P.A,93.90741
Val IOU,0.0
Val Loss,0.07484


Test Loss : 0.06999 | Test P.A : 99.21% | Test IOU : 0.00000 | 

Test IOU,▁
Test Loss,▁
Test P.A,▁
Test IOU,0.0
Test Loss,0.06999
Test P.A,99.21344
